In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
# Read in all relevant datasources

# CFB data
cfb2019 = pd.read_csv("/Users/wesdunphy/Documents/Code/Python/MAB/CFB2019.csv")

# Coaches Data
coaches = pd.read_csv("/Users/wesdunphy/Documents/Code/Python/MAB/Coaches9.csv")

# College Stadiums
# Include thousands arg to account for parsing the stadium volumes correctly
stadiums = pd.read_csv("/Users/wesdunphy/Documents/Code/Python/MAB/college_stadium_capacity.csv", thousands=',')

# Grad rates
grad_rates = pd.read_csv("/Users/wesdunphy/Documents/Code/Python/MAB/grad_rates.csv")

# University revenues
revenue_sources = pd.read_csv("/Users/wesdunphy/Documents/Code/Python/MAB/revenue_by_university.csv")

In [3]:
# Clean columns as necessary

# Remove dollar values and cast as floats
# Need to handle '--' cells as well
coaches.iloc[0:, 3:] = coaches.iloc[0:,3:].apply(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).apply(lambda x: x.str.replace('--','0')).astype(np.int64)

# Revenue needs to remove NaN values in dollar columns
# Apply same cleaning as coaches revenue
revenue_sources.dropna(inplace=True)
revenue_sources.iloc[0:, 5:] = revenue_sources.iloc[0:, 5:].apply(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).apply(lambda x: x.str.replace('--','0')).astype(np.int64)



### What is the best way to determine the salary of a coach?

Using available data for coaches, graduation rates, and revenues from stadiums, what is an appopriate salary for a coach?

Key questions we will want to answer:
 - What is an appropriate performance expectation given a coaches salary?
 - What, if any, relationship should there be between a coaches salary and graduation rate?
 - At what point is a coaches salary an unnecessary burden on a schools finances?

In [4]:
# We will want to join a dataframe of the cleaned raw datasets

# Isolate columns that we will want to keep

# Coaches colums
# Drop assistant column since they are all blank
coaches_sub = coaches[["School", "Conference", "Coach", "SchoolPay", "TotalPay", "Bonus", "BonusPaid", "Buyout"]]

# Football records
# Split "Team" into "School" and "Conference" columns
# Have to split on extra column to account for Miami(FL) and Miami(OH)
# These will have to merged with the School name and converted to match with coaches
cfb2019[["School", "Conference", "Extra"]] = cfb2019.Team.str.split("(", expand=True)
cfb2019[cfb2019["Extra"].isnull() == False] # identifies the miami teams
cfb2019.iat[57, -3] = "Miami (Fla.)" # matches coaches "School"
cfb2019.iat[58, -3] = "Miami (Ohio)" # matches coaches "School"
cfb2019 = cfb2019.drop(columns = ["Extra", "Conference"])
cfb2019_sub = cfb2019[["School", "Games", "Win-Loss", "Off Rank", "Def Rank"]]

# Grad Rates
# Only need school name + GSR/FGR
grad_rates_sub = grad_rates[["University", "GSR", "FGR"]]
grad_rates_sub.rename(columns={"University": "School"})

# Stadiums
stadiums_sub = stadiums[["College", "Capacity"]]
stadiums_sub.rename(columns={"College": "School"})

# Revenue Sources
revenue_sources_sub = revenue_sources[["College", 
                                       "Other Revenue",
                                       "Corporate Sponsorship, Advertising, Licensing",
                                       "Donor Contributions",
                                       "Competition Guarantees",
                                       "NCAA/Conference Distributions, Media Rights, and Post-Season Football",
                                       "Ticket Sales",
                                       "Institutional/Government Support",
                                       "Student Fees"]]
revenue_sources_sub.rename(columns={"College": "School"})

,School,Other Revenue,"Corporate Sponsorship, Advertising, Licensing",Donor Contributions,Competition Guarantees,"NCAA/Conference Distributions, Media Rights, and Post-Season Football",Ticket Sales,Institutional/Government Support,Student Fees
4,Arizona State University,3420533,3860104,11791138,1164271,8238025,10254504,11539986,0
5,Arkansas State University,731964,392271,502438,654320,700444,1464000,1027282,2348941
6,Auburn University,4056519,1110444,26306262,825480,13279251,20099721,0,922248
8,Ball State University,947104,226643,1522225,1103500,775635,1008701,3515736,7093120
10,Boise State University,1605792,1809906,2771494,767850,1748264,3845490,5643069,2401225
11,Bowling Green State University,1342963,594163,4549215,620000,727285,1009163,148873,8624189
14,California State University-Fresno,1409646,1788240,6558366,777500,2339874,8545065,5423735,254098
20,Central Michigan University,1628137,463849,971664,619450,944200,614863,12517504,0
23,Clemson University,4226818,2733825,8830649,1394402,11173528,14716363,1947677,1406844
28,Colorado State University,2309316,1847380,2737868,702115,2281989,2748331,4207588,2516159


In [ ]:
cfb2019_sub["School"].replace({"School": school_dict}, inplace=True)

In [ ]:
school_dict = {
    'UAB' : 'Alabama at Birmingham',
    'Appalachian St.' : 'Appalachian State',
    'Arizona St.' : 'Arizona State',
    'Arkansas St.' : 'Arkansas State',
    'Army West Point' : 'Army',
    'Ball St.' : 'Ball State',
    'Boise St.' : 'Boise State',
    'BYU' : 'Brigham Young',
    'UCF' : 'Central Florida',
    'Central Mich.' : 'Central Michigan',
    'Colorado St.' : 'Colorado State',
    'UConn' : 'Connecticut',
    'FIU' : 'Florida Internationa',
    'Fla. Atlantic' : 'Florida Atlantic',
    'Florida St.' : 'Florida State',
    'Fresno St.' : 'Fresno State',
    'Ga. Southern' : 'Georgia Southern',
    'Georgia St.' : 'Georgia State',
    'Iowa St.' : 'Iowa State',
    'Kansas St.' : 'Kansas State',
    'Kent St.' : 'Kent State',
    'La.-Monroe' : 'Louisiana-Monroe',
    'Michigan St.' : 'Michigan State',
    'Middle Tenn.': 'Middle Tennessee',
    'Mississippi St.' : 'Mississippi State',
    'NC State' : 'North Carolina State',
    'UNLV' : 'Nevada-Las Vegas',
    'New Mexico St.' : 'New Mexico State',
    'Northern Ill.' : 'Northern Illinois',
    'Ohio St.' : 'Ohio State',
    'Oklahoma St.' : 'Oklahoma State',
    'Oregon St.' : 'Oregon State',
    'Penn St.' : 'Penn State',
    'San Diego St.' : 'San Diego State',
    'San Jose St.' : 'San Jose State',
    'SMU' : 'Southern Methodist',
    'South Fla.' : 'South Florida',
    'Southern Miss.' : 'Southern Mississippi',
    'Texas St.' : 'Texas State',
    'TCU' : 'Texas Christian',
    'UTEP' : 'Texas El-Paso',
    'UTSA' : 'Texas San Antonio',
    'Utah St.' : 'Utah State',
    'Washington St.' : 'Washington State',
    'Western Ky.' : 'Western Kentucky',
    'Western Mich.' : 'Western Michigan',
}

In [6]:
cfb2019_sub

,School,Games,Win-Loss,Off Rank,Def Rank
0,Air Force,13,11-2,51,17
1,Akron,12,0-12,130,83
2,Alabama,13,11-2,6,20
3,Appalachian St.,14,13-1,39,26
4,Arizona,12,4-8,30,120
5,Arizona St.,13,8-5,94,67
6,Arkansas,12,2-10,111,110
7,Arkansas St.,13,8-5,33,124
8,Army West Point,13,5-8,89,30
9,Auburn,13,9-4,64,28
